In [7]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import scipy
import importlib
import rsc 
import SatCallbacks
import Clone_Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

import pkgutil

"""
(trainX, trainy), (testX, testy) = rsc.get_cifar10()

# load dataset
#(trainX, trainy), (testX, testy) = cifar10.load_data()
# summarize loaded dataset
print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
print('Test: X=%s, y=%s' % (testX.shape, testy.shape))
# plot first few images
for i in range(9):
	# define subplot
	plt.subplot(330 + 1 + i)
	# plot raw pixel data
	plt.imshow(trainX[i])
# show the figure
plt.show()
"""

(train_images, train_labels), (test_images, test_labels) = rsc.get_cifar10(one_hot=False)

Shape of cifar10 y_train: (50000, 1)


In [8]:
#   CIFAR-10 ON EXAMPLE MODEL
######################################

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

#model.summary()

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=1, batch_size=7, 
                    validation_data=(test_images, test_labels))



7143/7143 [==============================] - 335s 47ms/step - loss: 1.5021 - accuracy: 0.4571 - val_loss: 1.2760 - val_accuracy: 0.5377


In [11]:
importlib.reload(Clone_Model)
importlib.reload(SatCallbacks)
#model.summary()


clone = Clone_Model.satify_model(model)

"""
clone.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              run_eagerly = tf.executing_eagerly())
"""
#model.summary()
#clone.summary()

#for l in clone.layers:
#    if l

# commented out 20.12.21
#sat_cb = SatCallbacks.sat_results()
#new_cb = SatCallbacks.sat_logger()

sat_cb = SatCallbacks.sat_callback()

history = clone.fit(train_images, train_labels,
                    epochs=1,
                    batch_size=7,
                    steps_per_epoch=3,
                    validation_data=(test_images, test_labels), 
                    validation_steps=1, 
                    callbacks=[sat_cb])



Model output_shape: (None, 10)
input_shape: [None, 64]
layer_width: (64,)
SatLayer input shape (None, 64) not matching current_activation property shape (0, 64)
batch_size initialized. current shape is: Tensor("dense_2/Shape:0", shape=(1,), dtype=int32)


ValueError: in user code:

    /home/leo/Desktop/Thesis/New_Repo/Clone_Model.py:152 call  *
        _ = self.log_layer(out)
    /home/leo/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:897 __call__  **
        self._maybe_build(inputs)
    /home/leo/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:2416 _maybe_build
        self.build(input_shapes)  # pylint:disable=not-callable
    /home/leo/Desktop/Thesis/New_Repo/SatLayer.py:612 build
        trainable=False)
    /home/leo/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/ops/variables.py:261 __call__
        return cls._variable_v2_call(*args, **kwargs)
    /home/leo/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/ops/variables.py:255 _variable_v2_call
        shape=shape)
    /home/leo/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/ops/variables.py:236 <lambda>
        previous_getter = lambda **kws: 

In [5]:
#     CIFAR-10 ON PRE-TRAINED VGG-16
#############################################


model = VGG16(weights='imagenet', include_top=False)
img = train_images[1]
print(np.shape(img))
img = tf.image.resize(img, [224,224])
print(np.shape(img))

#plt.imshow(img)



#img_input = tf.keras.Input(tensor=img, shape=[None,224,224,3])
output = model.predict(imgs)


#img = cifar


(32, 32, 3)
(224, 224, 3)


ValueError: in user code:

    /home/leo/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    /home/leo/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/leo/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/leo/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/leo/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1122 predict_step  **
        return self(x, training=False)
    /home/leo/anaconda3/envs/tf/lib/python3.7/site-packages/tensorf

In [1]:
#   NEW END-PHASE TRIAL
#####################################

cb_dict = {'track_naive': False,
          'track_two_pass': True}

sat_cb = sat_callback(instruction_dict)

NameError: name 'sat_callback' is not defined